In [37]:
# requirements for this task to run
# Pandas
# Gensim
# Sklearn

import io, os, sys, types, time, datetime, math, random, requests, subprocess, tempfile
import gensim.models.keyedvectors
import keras
import numpy as np

# -----------------------
# set up task options
#------------------------

MAX_NUM_WORDS = 10000
EMB_DIMS = 50

sen1 = "how are you"
sen2 = "nice to meet you and good day"

# ----------------------
# helper functions
# ----------------------

def sentence_avg(sentence):
  # A list to aggreate
  sum = []
  keylist = list(word_index.keys())
  for word in sentence:
    word = keylist[word-1]
    if word in w2v_gl:
      sum.append(w2v_gl[word])
  sum = np.array(sum)
  return sum.mean(0)

# Note when call fit_on_texts, it must be entire corpus, not a sentence
tokenizer = keras.preprocessing.text.Tokenizer(num_words = MAX_NUM_WORDS)
corpus = (sen1, sen2)
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

from gensim.models.word2vec import Word2Vec
PRE_TRAINED_EMBEDDING = '50d.w2vformat.txt'
model_g = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(PRE_TRAINED_EMBEDDING, binary = False)
w2v_gl = {w: vec for w, vec in zip((model_g.vocab), model_g.vectors)}
print(w2v_gl['word'])
print(X)

print(sentence_avg(X[0], EMB_DIMS))

[-0.1643     0.15722   -0.55021   -0.3303     0.66463   -0.1152
 -0.2261    -0.23674   -0.86119    0.24319    0.074499   0.61081
  0.73683   -0.35224    0.61346    0.0050975 -0.62538   -0.0050458
  0.18392   -0.12214   -0.65973   -0.30673    0.35038    0.75805
  1.0183    -1.7424    -1.4277     0.38032    0.37713   -0.74941
  2.9401    -0.8097    -0.66901    0.23123   -0.073194  -0.13624
  0.24424   -1.0129    -0.24919   -0.06893    0.70231   -0.022177
 -0.64684    0.59599    0.027092   0.11203    0.61214    0.74339
  0.23572   -0.1369   ]
[[2, 3, 1], [4, 5, 6, 1, 7, 8, 9]]
[ 5.50072670e-01  7.97720030e-02  2.48529986e-01 -3.27209979e-01
  6.13483310e-01 -5.24228401e-02 -4.36006635e-01 -2.67500937e-01
 -2.83643335e-01  9.96333361e-02 -2.75326997e-01  6.30840003e-01
 -1.77470013e-01  1.13804005e-01  8.46876621e-01  6.11603320e-01
  5.24820030e-01  7.79526606e-02  3.33656669e-01 -1.14258659e+00
 -3.31656665e-01  2.83790022e-01  8.38809967e-01  4.56220031e-01
  6.09003305e-01 -1.72724664e

# Load Raw csv
Here we try to import the data and load it into a Pandas data frame object

In [13]:
!ls

datafiles  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
datalab    glove.6B.200d.txt  glove.6B.50d.txt


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [0]:
!unzip glove.6B.zip

In [15]:
# This steps required only if the file has not been in word2vec format
!pip install -q gensim
!python -m gensim.scripts.glove2word2vec --input glove.6B.50d.txt --output 50d.w2vformat.txt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'gensim.scripts.glove2word2vec' found in sys.modules after import of package 'gensim.scripts', but prior to execution of 'gensim.scripts.glove2word2vec'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2018-06-20 04:54:00,737 - glove2word2vec - INFO - running /usr/local/lib/python3.6/dist-packages/gensim/scripts/glove2word2vec.py --input glove.6B.50d.txt --output 50d.w2vformat.txt
2018-06-20 04:54:08,250 - glove2word2vec - INFO - converting 400000 vectors from glove.6B.50d.txt to 50d.w2vformat.txt
2018-06-20 04:54:09,097 - glove2word2vec - INFO - Converted model with 400000 vectors and 50 dimensions


In [21]:
# embedding file for glove.6B.50D, replace with your file
from gensim.models.word2vec import Word2Vec
PRE_TRAINED_EMBEDDING = '50d.w2vformat.txt'
model_g = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(PRE_TRAINED_EMBEDDING, binary = False)
w2v_gl = {w: vec for w, vec in zip((model_g.vocab), model_g.vectors)}
print(w2v_gl['word'])

[-0.1643     0.15722   -0.55021   -0.3303     0.66463   -0.1152
 -0.2261    -0.23674   -0.86119    0.24319    0.074499   0.61081
  0.73683   -0.35224    0.61346    0.0050975 -0.62538   -0.0050458
  0.18392   -0.12214   -0.65973   -0.30673    0.35038    0.75805
  1.0183    -1.7424    -1.4277     0.38032    0.37713   -0.74941
  2.9401    -0.8097    -0.66901    0.23123   -0.073194  -0.13624
  0.24424   -1.0129    -0.24919   -0.06893    0.70231   -0.022177
 -0.64684    0.59599    0.027092   0.11203    0.61214    0.74339
  0.23572   -0.1369   ]


In [0]:
# Psesudo code only, don't work 
DATASET = (
  "http://any.com/my/data.file1",
  "http://any.com/my/data.file2",
)

#Download the embedding vectors from glove
def download_data(path='datafiles', urls=DATASET):
    if not os.path.exists(path):
        os.mkdir(path)
        
    for url in urls:
        response = requests.get(url)
        name = os.path.basename(url)
        with open(os.path.join(path, name), 'w') as f:
            f.write(response.content.decode('utf-8'))

            
download_data()

In [0]:
import tensorflow as tf

def download_glove():
  em_file = 'glove.6B.zip'
  tf.keras.utils.get_file(
      fname=em_file,
      
    )